### Importing the dataset

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import joblib

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
df = pd.read_csv('Dataset/XWines_with_Ratings.csv', low_memory=False)

In [5]:
df.head()

,RatingID,UserID,Rating,WineName,Vintage,ABV,Body,Acidity,Country,RegionName,WineryName,Harmonize1,Harmonize2,Harmonize3,Harmonize4,Harmonize5,Harmonize6,Harmonize7,Harmonize8,Harmonize9,Harmonize10,Harmonize11,Harmonize12,Grapes1,Grapes2,Grapes3,Grapes4,Grapes5,Grapes6,Grapes7,Grapes8,Grapes9,Elaborate1,Elaborate2,Type1,Type2
0,326545,1756594,4.0,Espumante Moscatel,1999,8.0,Mediumbodied,High,Brazil,Serra Gaúcha,Casa Perini,Pork,Rich Fish,Shellfish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muscat/Moscato,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Varietal,100%,Sparkling,NaN
1,1314107,1219305,2.5,Espumante Moscatel,2007,8.0,Mediumbodied,High,Brazil,Serra Gaúcha,Casa Perini,Pork,Rich Fish,Shellfish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muscat/Moscato,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Varietal,100%,Sparkling,NaN
2,1446366,2047929,3.5,Espumante Moscatel,2008,8.0,Mediumbodied,High,Brazil,Serra Gaúcha,Casa Perini,Pork,Rich Fish,Shellfish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muscat/Moscato,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Varietal,100%,Sparkling,NaN
3,1448872,1006545,5.0,Espumante Moscatel,2008,8.0,Mediumbodied,High,Brazil,Serra Gaúcha,Casa Perini,Pork,Rich Fish,Shellfish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muscat/Moscato,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Varietal,100%,Sparkling,NaN
4,1657104,1400823,2.0,Espumante Moscatel,2008,8.0,Mediumbodied,High,Brazil,Serra Gaúcha,Casa Perini,Pork,Rich Fish,Shellfish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muscat/Moscato,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Varietal,100%,Sparkling,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 36 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   RatingID     150000 non-null  int64  
 1   UserID       150000 non-null  int64  
 2   Rating       150000 non-null  float64
 3   WineName     150000 non-null  object 
 4   Vintage      150000 non-null  object 
 5   ABV          150000 non-null  float64
 6   Body         150000 non-null  object 
 7   Acidity      150000 non-null  object 
 8   Country      150000 non-null  object 
 9   RegionName   150000 non-null  object 
 10  WineryName   150000 non-null  object 
 11  Harmonize1   150000 non-null  object 
 12  Harmonize2   149778 non-null  object 
 13  Harmonize3   146425 non-null  object 
 14  Harmonize4   108107 non-null  object 
 15  Harmonize5   30055 non-null   object 
 16  Harmonize6   12094 non-null   object 
 17  Harmonize7   1109 non-null    object 
 18  Harmonize8   670 non-nul

### Combined the variables into one column

In [7]:
df = df.fillna("")

In [8]:
common_columns = ['WineName', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName']
grapes_columns = [f'Grapes{i}' for i in range(1, 10)]
harmonize_columns = [f'Harmonize{i}' for i in range(1, 13)]
type_columns = [f'Type{i}' for i in range(1, 2)]
elaborate_columns = [f'Elaborate{i}' for i in range(1, 2)]
all_columns = common_columns + grapes_columns + harmonize_columns + type_columns + elaborate_columns
df['Attributes'] = df[all_columns].astype(str).agg(' '.join, axis=1)

In [9]:
df['Attributes'].head()

0    Espumante Moscatel Mediumbodied High Brazil Serra Gaúcha Casa Perini Muscat/Moscato         Pork Rich Fish Shellfish          Sparkling Varietal
1    Espumante Moscatel Mediumbodied High Brazil Serra Gaúcha Casa Perini Muscat/Moscato         Pork Rich Fish Shellfish          Sparkling Varietal
2    Espumante Moscatel Mediumbodied High Brazil Serra Gaúcha Casa Perini Muscat/Moscato         Pork Rich Fish Shellfish          Sparkling Varietal
3    Espumante Moscatel Mediumbodied High Brazil Serra Gaúcha Casa Perini Muscat/Moscato         Pork Rich Fish Shellfish          Sparkling Varietal
4    Espumante Moscatel Mediumbodied High Brazil Serra Gaúcha Casa Perini Muscat/Moscato         Pork Rich Fish Shellfish          Sparkling Varietal
Name: Attributes, dtype: object

In [10]:
df.head()

,RatingID,UserID,Rating,WineName,Vintage,ABV,Body,Acidity,Country,RegionName,WineryName,Harmonize1,Harmonize2,Harmonize3,Harmonize4,Harmonize5,Harmonize6,Harmonize7,Harmonize8,Harmonize9,Harmonize10,Harmonize11,Harmonize12,Grapes1,Grapes2,Grapes3,Grapes4,Grapes5,Grapes6,Grapes7,Grapes8,Grapes9,Elaborate1,Elaborate2,Type1,Type2,Attributes
0,326545,1756594,4.0,Espumante Moscatel,1999,8.0,Mediumbodied,High,Brazil,Serra Gaúcha,Casa Perini,Pork,Rich Fish,Shellfish,,,,,,,,,,Muscat/Moscato,,,,,,,,,Varietal,100%,Sparkling,,Espumante Moscatel Mediumbodied High Brazil Serra Gaúcha Casa Perini Muscat/Moscato Pork Rich Fish Shellfish Sparkling Varietal
1,1314107,1219305,2.5,Espumante Moscatel,2007,8.0,Mediumbodied,High,Brazil,Serra Gaúcha,Casa Perini,Pork,Rich Fish,Shellfish,,,,,,,,,,Muscat/Moscato,,,,,,,,,Varietal,100%,Sparkling,,Espumante Moscatel Mediumbodied High Brazil Serra Gaúcha Casa Perini Muscat/Moscato Pork Rich Fish Shellfish Sparkling Varietal
2,1446366,2047929,3.5,Espumante Moscatel,2008,8.0,Mediumbodied,High,Brazil,Serra Gaúcha,Casa Perini,Pork,Rich Fish,Shellfish,,,,,,,,,,Muscat/Moscato,,,,,,,,,Varietal,100%,Sparkling,,Espumante Moscatel Mediumbodied High Brazil Serra Gaúcha Casa Perini Muscat/Moscato Pork Rich Fish Shellfish Sparkling Varietal
3,1448872,1006545,5.0,Espumante Moscatel,2008,8.0,Mediumbodied,High,Brazil,Serra Gaúcha,Casa Perini,Pork,Rich Fish,Shellfish,,,,,,,,,,Muscat/Moscato,,,,,,,,,Varietal,100%,Sparkling,,Espumante Moscatel Mediumbodied High Brazil Serra Gaúcha Casa Perini Muscat/Moscato Pork Rich Fish Shellfish Sparkling Varietal
4,1657104,1400823,2.0,Espumante Moscatel,2008,8.0,Mediumbodied,High,Brazil,Serra Gaúcha,Casa Perini,Pork,Rich Fish,Shellfish,,,,,,,,,,Muscat/Moscato,,,,,,,,,Varietal,100%,Sparkling,,Espumante Moscatel Mediumbodied High Brazil Serra Gaúcha Casa Perini Muscat/Moscato Pork Rich Fish Shellfish Sparkling Varietal


### Remove duplicated rows in Attributes and WineName

In [11]:
display(f'Duplicated in Attributes: {df["Attributes"].duplicated().sum()}')
display(f'Duplicated in WineName: {df["WineName"].duplicated().sum()}')
display(f'Shape: {df.shape}')

'Duplicated in Attributes: 148993'

'Duplicated in WineName: 149197'

'Shape: (150000, 37)'

In [12]:
df.drop_duplicates(subset=['Attributes'], inplace=True)
df.drop_duplicates(subset=['WineName'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [26]:
display(f'Duplicated in Attributes: {df["Attributes"].duplicated().sum()}')
display(f'Duplicated in WineName: {df["WineName"].duplicated().sum()}')
display(f'Shape: {df.shape}')
df.head()

'Duplicated in Attributes: 0'

'Duplicated in WineName: 0'

'Shape: (803, 37)'

,RatingID,UserID,Rating,WineName,Vintage,ABV,Body,Acidity,Country,RegionName,WineryName,Harmonize1,Harmonize2,Harmonize3,Harmonize4,Harmonize5,Harmonize6,Harmonize7,Harmonize8,Harmonize9,Harmonize10,Harmonize11,Harmonize12,Grapes1,Grapes2,Grapes3,Grapes4,Grapes5,Grapes6,Grapes7,Grapes8,Grapes9,Elaborate1,Elaborate2,Type1,Type2,Attributes
0,326545,1756594,4.0,Espumante Moscatel,1999,8.0,Mediumbodied,High,Brazil,Serra Gaúcha,Casa Perini,Pork,Rich Fish,Shellfish,,,,,,,,,,Muscat/Moscato,,,,,,,,,Varietal,100%,Sparkling,,Espumante Moscatel Mediumbodied High Brazil Serra Gaúcha Casa Perini Muscat/Moscato Pork Rich Fish Shellfish Sparkling Varietal
1,2103556,1291385,4.5,Ancellotta,2009,12.0,Mediumbodied,Medium,Brazil,Serra Gaúcha,Casa Perini,Beef,Barbecue,Codfish,Pasta,Pizza,Cheese,,,,,,,Ancellotta,,,,,,,,,Varietal,100%,Red,,Ancellotta Mediumbodied Medium Brazil Serra Gaúcha Casa Perini Ancellotta Beef Barbecue Codfish Pasta Pizza Cheese Red Varietal
2,4116770,1195131,3.0,Cabernet Sauvignon,2011,12.0,Fullbodied,High,Brazil,Serra Gaúcha,Castellamare,Beef,Lamb,Poultry,,,,,,,,,,Cabernet Sauvignon,,,,,,,,,Varietal,100%,Red,,Cabernet Sauvignon Fullbodied High Brazil Serra Gaúcha Castellamare Cabernet Sauvignon Beef Lamb Poultry Red Varietal
3,1523098,1640888,3.0,Maison de Ville Cabernet Merlot,2008,11.0,Fullbodied,Medium,Brazil,Serra Gaúcha,Aurora,Beef,Lamb,Game Meat,Poultry,,,,,,,,,Cabernet Sauvignon,Merlot,,,,,,,,Assemblage,Bordeaux Red Blend,Red,,Maison de Ville Cabernet Merlot Fullbodied Medium Brazil Serra Gaúcha Aurora Cabernet Sauvignon Merlot Beef Lamb Game Meat Poultry Red Assemblage
4,5667238,2034035,3.0,Do Lugar Moscatel Espumantes,2012,8.0,Mediumbodied,High,Brazil,Serra Gaúcha,Dal Pizzol,Pork,Rich Fish,Shellfish,,,,,,,,,,Muscat/Moscato Bianco,,,,,,,,,Varietal,100%,Sparkling,,Do Lugar Moscatel Espumantes Mediumbodied High Brazil Serra Gaúcha Dal Pizzol Muscat/Moscato Bianco Pork Rich Fish Shellfish Sparkling Varietal


### Converting 'Attributes' into a TF-IDF Matrix

In [14]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['Attributes'])
tfidf_matrix.shape

(803, 2167)

### Computing Cosine Similarity between Wines based on TF-IDF Matrix

In [16]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.33618266, 0.24266187, ..., 0.0761952 , 0.04466483,
        0.00511421],
       [0.33618266, 1.        , 0.19515172, ..., 0.00491391, 0.01095248,
        0.04383445],
       [0.24266187, 0.19515172, 1.        , ..., 0.03867137, 0.01826841,
        0.03813597],
       ...,
       [0.0761952 , 0.00491391, 0.03867137, ..., 1.        , 0.12663979,
        0.01230003],
       [0.04466483, 0.01095248, 0.01826841, ..., 0.12663979, 1.        ,
        0.00334867],
       [0.00511421, 0.04383445, 0.03813597, ..., 0.01230003, 0.00334867,
        1.        ]])

In [17]:
def get_recommendations(WineName):

    recommendation = {}

    idx = df.index[df['WineName'] == WineName].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    wine_indices = [i[0] for i in sim_scores]

    recommendation = {
        'WineName': df['WineName'].iloc[wine_indices],
        'Score': [i[1] for i in sim_scores]
    }

    return pd.DataFrame(recommendation)

In [24]:
name = df['WineName'].sample(1).values[0]

'Oloroso V O R S'

In [25]:
display(f'Recommendation for: {name}')
get_recommendations(name)

'Recommendation for: Oloroso V O R S'

,WineName,Score
88,Medium Dry Sherry,0.653194
109,Don Fino Superior Fino Sherry,0.392337
473,Rare Cream Sherry Superior Solera Reserva,0.377408
110,Character Medium Dry Amontillado Sherry,0.341123
469,Pedro Ximénez Murillo Selección del Centenario,0.316428
702,Triple Cream Sherry,0.279645
435,Grappa Riserva Botti da Sherry,0.220397
470,A Galet Rosado,0.175442
189,Blanc,0.174152
699,Port,0.170814


### Exporting the model

In [16]:
dummies = pd.get_dummies(df.WineName, dtype=int)
dummies.head(3)

,10 Anos Tawny Porto,10 Year Old Tawny Port,10 Year Old Tawny Porto,101 Moscato d Asti,12 e Mezzo Primitivo del Salento,20 Anos Tawny Porto,20 Year Old Tawny Port,20 Years Old Tawny Porto,21 Gables Chenin Blanc,A Galet Rosado,A Rosé Is A Rosé Is A Rosé,AME Cabernet Sauvignon,ARTEMIS Cabernet Sauvignon,Abednego Shiraz Grenache Mourvedre,Acquasantiera Tinto Seco Fino,Adobe Carmenère Reserva,Agnus Cabernet Sauvignon,Al Poggio Chardonnay di Toscana,Alegria de Hérencia del Padrí Priorat,Alentejo Tinto,Alma Mora Cabernet Sauvignon,Altos del Plata Malbec,Alvarinho,Além do Rio Branco,Amarone della Valpolicella,Amarone della Valpolicella Classico,Ancellotta,Anderson Valley Rosé of Pinot Noir,Angelin Langhe Nebbiolo,Applejack Vineyard Pinot Noir,Aquarela Moscatel Espumante,Aquarius Ranch Vineyard Chardonnay,Aragonêz Rosé,Arborina Barolo,Art Series Chardonnay,Assyrtiko,Aszú 6 Puttonyos,Atempo Rosé,Australia Felix Chardonnay Viognier,Australia Felix Sagrantino,Bairrada Reserva Da Familia,Bandol Cuvée Spéciale La Louffe,Barbaresco,Barbera,Barbera D Alba,Barbera d Alba,Barbera d Alba Superiore,Barbera d Asti,Barbera d’Alba Pilade,Bardolino Classico,Barolo,Barolo Tradizione,Barolo Arborina,Barolo Bussia,Barolo Marcenasco,Barolo Riserva,Barolo Vigneto Cannubi,Barrel Select Fumé Blanc,Beaujolais,Beaujolais Gamay,Beaujolais Villages,Belcaro Vin Santo del Chianti Classico,Bendigo Vineyard Pinot Noir,Benjamin Malbec,Bermatinger Leopoldsberg B Spätburgunder,Bernkasteler Badstube Riesling Spätlese,Bin 343 Cabernet Sauvignon Shiraz,Black Label Nik Weis Saar Riesling,Blackwood Park Botrytis Riesling,Blanc,Blanc de Blancs,Blanc de Blancs Brut Champagne,Blanc de Blancs Brut Champagne Grand Cru Chouilly,Blanc de Blancs Extra Brut Champagne Grand Cru Le Mesnil sur Oger,Blanc de Blancs Millésime Prestige Brut Champagne Grand Cru,Blanco,Blason de France Champagne,Blauer Spätburgunder Trocken,Blu Prosecco Extra Dry,Blue Burg Layer Schlosskapelle Kerner Spätlese,Blush Syrah Viognier,Bobal Tierra de Castilla,Bordeaux,Bordeaux Rouge,Bordeaux Sauvignon Blanc,Bordeaux Supérieur,Bourgogne Aligoté,Bourgogne Chardonnay,Bourgogne Couvent des Jacobins,Bourgogne Cuvée Latour,Bourgogne Gamay,Bourgogne Pinot Noir,Bourgogne Pinot Noir La Vignée,Bourgogne Pinot Noir Vieilles Vignes,Brachetto d Acqui,Branco,Brauneberger Juffer Riesling Auslese,Bricco Francesco Rocche dell Annunziata Barolo,Brunello di Montalcino,Brut,Brut Blanc de Blancs,Brut R de Ruinart Champagne,Brut Champagne,Brut Classic,Brut Nature Silver Champagne Grand Cru Bouzy,Brut Prosecco,Brut Rosé,Brut Vintage,Brut Vintage Champagne Extra Cuvée de Réserve,Burgenland Red,Burnside Vineyard Chardonnay,Bush Vine Pinotage,Cabardès Rosé,Cabernet Franc,Cabernet Franc Rosé,Cabernet Sauvignon,Cabernet Sauvignon Aged in Bourbon Barrels,Cabernet Sauvignon Merlot,Cabernet Sauvignon Rosado,Cabernet Sauvignon Rosé,Cabernet Sauvignon Shiraz,Cahors Malbec,Caillou Blanc,Calda Bordaleza Tinto,California Champagne Brut,Calixa Syrah,Camins del Priorat,Campo Largo Branco Seco de Mesa,Cane Cut,Cannubi San Lorenzo Ravera Barolo,Carmenere,Carmenère,Carmenère Gran Reserva,Carmenérè,Caroline Graves,Carrasviñas Rueda Verdejo,Casa Burmester Touriga Nacional,Cava Brut,Cava Clasico Brut,Cava Gran Reserva Brut Nature Reserva de la Familia,Cave Amadeu Rosé Brut,Cave Cabernet Sauvignon,Cedro do Noval,Cellar Selection Cabernet Franc Rosé,Cellar Selection Sauvignon Blanc,Cellar Selection Syrah,Cerequio Barolo,Chablis,Chablis Champs Royaux,Chablis 1er Cru Montmains,Chablis Cellier de La Sablière,Chambolle Musigny,Character Medium Dry Amontillado Sherry,Chardonnay,Chardonnay Bourgogne Les Ursulines,Chardonnay Unfiltered,Chasselas,Chateau Porto Carras,Chenin Blanc,Chenin Blanc Cosecha Tardía,Chenin Chardonnay Roca,Chianti,Chianti Classico,Chianti Classico Riserva,Chianti Colli Senesi,Chianti Riserva,Christina Ribera del Duero,Château Chemin Royal Moulis en Médoc,Châteauneuf du Pape,Cinsault Grenache Rosé,Cinsault Rosé,Clarette Rosé Trocken,Clasico

In [17]:
import json
columns = {
    'data_columns' : [col.lower() for col in dummies.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))

In [18]:
df.head(1)

,RatingID,UserID,Rating,WineName,Vintage,ABV,Body,Acidity,Country,RegionName,WineryName,Harmonize1,Harmonize2,Harmonize3,Harmonize4,Harmonize5,Harmonize6,Harmonize7,Harmonize8,Harmonize9,Harmonize10,Harmonize11,Harmonize12,Grapes1,Grapes2,Grapes3,Grapes4,Grapes5,Grapes6,Grapes7,Grapes8,Grapes9,Elaborate1,Elaborate2,Type1,Type2,Attributes
0,326545,1756594,4.0,Espumante Moscatel,1999,8.0,Mediumbodied,High,Brazil,Serra Gaúcha,Casa Perini,Pork,Rich Fish,Shellfish,,,,,,,,,,Muscat/Moscato,,,,,,,,,Varietal,100%,Sparkling,,Espumante Moscatel Mediumbodied High Brazil Serra Gaúcha Casa Perini Muscat/Moscato Pork Rich Fish Shellfish Sparkling Varietal


In [19]:
df['WineName'] = df['WineName'].str.lower()

In [20]:
joblib.dump((cosine_sim, df), 'content_based_recommender_model.pkl')

['content_based_recommender_model.pkl']